In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.listdir("/content/drive/MyDrive")



['Colab Notebooks', 'lcd.zip', 'LiverCancerProject']

In [5]:
zip_path =  '/content/drive/MyDrive/LiverCancerProject/img.zip'


In [6]:
import zipfile

zip_path = '/content/drive/MyDrive/LiverCancerProject/img.zip'# <-- use your real path
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/images')


In [7]:
image_folder = '/content/images/img'
images = [f for f in os.listdir(image_folder) if f.lower().endswith('.png')]
print(f"Total images: {len(images)}")

Total images: 611


In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Input, Model
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import cv2
import glob

# Load CSV files
df_train = pd.read_csv('/content/lits_train.csv')
df_test = pd.read_csv('/content/lits_test.csv')

# Folder where images are stored
image_folder = '/content/images/img'  # <-- Change to your folder path
dataset_paths = sorted(glob.glob(os.path.join(image_folder, '*.png')))

# Define the load_images function
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img.shape != (128, 128):
            img = cv2.resize(img, (128, 128))
        images.append(img)
    return np.array(images).reshape(-1, 128, 128, 1)

# Load dataset
X = load_images(dataset_paths)
X = X / 255.0

# Extract labels
y = df_train['tumor_mask_empty'].values[:len(X)]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# CNN model
inputs = Input(shape=(128, 128, 1))
x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

cnn_model = Model(inputs=inputs, outputs=outputs)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train CNN
cnn_model.fit(X_train, y_train, epochs=10, batch_size=4, validation_split=0.2)

# Extract CNN features
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# XGBoost on CNN features
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb_model.fit(X_train_features, y_train.ravel())
y_pred = xgb_model.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
print(f'XGBoost Accuracy: {accuracy:.4f}')

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.9465 - loss: 0.8782 - val_accuracy: 0.9592 - val_loss: 0.2157
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9173 - loss: 0.3819 - val_accuracy: 0.9592 - val_loss: 0.3824
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9521 - loss: 0.2922 - val_accuracy: 0.9592 - val_loss: 0.1761
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9678 - loss: 0.2328 - val_accuracy: 0.9592 - val_loss: 0.2874
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9392 - loss: 0.2412 - val_accuracy: 0.9592 - val_loss: 0.1667
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9400 - loss: 0.2348 - val_accuracy: 0.9592 - val_loss: 0.1641
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9644 - loss: 0.1684 - val_accuracy: 0.9592 - val_loss: 0.1575
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9597 - loss: 0.1712 - val_accuracy: 0.9592 - v

In [9]:
feature_extractor.save('/content/feature_extractor_model.h5')

In [10]:
feature_extractor = tf.keras.models.load_model('/content/feature_extractor_model.h5')

In [11]:
# Save your trained models (you should have already done this)
feature_extractor.save('/content/feature_extractor_model.h5')

import joblib
joblib.dump(xgb_model, '/content/xgb_model.pkl')

['/content/xgb_model.pkl']

In [12]:
!pip install streamlit pyngrok xgboost opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [13]:
import os
os.environ["NGROK_AUTHTOKEN"] = "2jobd681TGhlxwJLHtUfB2JfBrp_7BBJHimf3bV2aJyti3B5K"

In [14]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import cv2
import glob
import os
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers, Input
from xgboost import XGBClassifier
import joblib

# Load trained models (you must save them after training in Colab)
cnn_model = load_model('/content/feature_extractor_model.h5')
xgb_model = joblib.load('/content/xgb_model.pkl')

# Feature extractor
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

def load_image(image_file):
    img = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))
    return img.reshape(1, 128, 128, 1) / 255.0

st.title("Tumor Detection using CNN + XGBoost")

uploaded_file = st.file_uploader("Upload an image", type=['png', 'jpg', 'jpeg'])
if uploaded_file is not None:
    st.image(uploaded_file, caption='Uploaded Image', width=250)
    img = load_image(uploaded_file)

    cnn_features = feature_extractor.predict(img)
    prediction = xgb_model.predict(cnn_features)[0]
    st.write("Prediction:", "Tumor Detected" if prediction == 1 else "No Tumor")

Writing app.py


In [15]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.219.225:8501

  Stopping...


In [16]:
import streamlit as st
import numpy as np
import cv2
import tensorflow as tf
import pickle

# Load models
cnn_model = tf.keras.models.load_model('/content/feature_extractor_model.h5')  # Make sure this file exists
with open("/content/xgb_model.pkl", "rb") as f:
    xgb_model = pickle.load(f)  # XGBoost model on CNN features

st.title("Liver Tumor Detection from CT Scan")

uploaded_file = st.file_uploader("Upload a Liver CT Scan Image", type=["png", "jpg", "jpeg"])

if uploaded_file is not None:
    # Display image
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_GRAYSCALE)

    # Preprocess
    img_resized = cv2.resize(img, (128, 128))
    img_input = img_resized.reshape(1, 128, 128, 1) / 255.0

    st.image(img, caption="Uploaded CT Scan", use_container_width=True)

    # Feature extraction from CNN
    feature_extractor = tf.keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
    cnn_features = feature_extractor.predict(img_input)

    # XGBoost Prediction
    prediction = xgb_model.predict(cnn_features)[0]
    label = "Tumor Detected" if prediction == 1 else "No Tumor Found"

    # Display result
    st.success(f"Prediction: {label}")

2025-04-25 05:28:33.117 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.260 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-25 05:28:33.262 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.264 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-25 05:28:33.267 Thread 'MainThread': mi

In [17]:
!pip install streamlit pyngrok pillow

In [18]:
!ngrok config add-authtoken 2w5cMSLJWZtB20oKypL38j8uJgj_7Ets2As4UPW5udBCnKJoY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Install required packages
!pip install streamlit pyngrok pillow

# Write the Streamlit app to a file
code = """
import streamlit as st
from PIL import Image
import numpy as np

def predict_tumor(image_array):
    return "Tumor Detected" if np.mean(image_array) < 100 else "No Tumor Detected"

st.title("Liver Cancer Detection from CT Scans")
st.write("Upload a CT scan image to detect presence of liver tumor.")

uploaded_file = st.file_uploader("Choose a CT scan image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('L')  # grayscale
    st.image(image, caption="Uploaded CT Image", use_column_width=True)

    image_resized = image.resize((224, 224))
    image_array = np.array(image_resized)

    prediction = predict_tumor(image_array)
    st.subheader("Prediction:")
    st.success(prediction)
    """
with open("app.py", "w") as f:
    f.write(code)

# Start ngrok
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

# Run Streamlit app
!streamlit run app.py --server.enableCORS false --server.enableXsrfProtection false &

Public URL: NgrokTunnel: "https://9f7c-34-16-219-225.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.219.225:8501

2025-04-25 05:30:35.421 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `use_container_width` parameter instead.
2025-04-25 05:30:47.539 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `use_container_width` parameter instead.
